In [ ]:
import sys
import math
from numbers import Number
import numpy as np
from numpy.testing import assert_allclose
import contextlib

from qiskit import QiskitError
from qiskit import QuantumRegister, QuantumCircuit
from qiskit.circuit.library import HGate, QFT

from qiskit.quantum_info import Statevector, DensityMatrix
from qiskit.quantum_info.random import random_unitary, random_pauli
from qiskit.circuit.library import QuantumVolume
from qiskit.quantum_info.operators.operator import Operator
from qiskit.quantum_info.operators.symplectic import Pauli, SparsePauliOp

from qiskit_aer.quantum_info.states import AerDensityMatrix, AerStatevector

In [ ]:
def rand_vec(n, normalize=False):
    """Return complex vector or AerStatevector"""
    seed = np.random.randint(0, np.iinfo(np.int32).max)
    rng = np.random.default_rng(seed)
    vec = rng.random(n) + 1j * rng.random(n)
    if normalize:
        vec /= np.sqrt(np.dot(vec, np.conj(vec)))
    return vec

def rand_rho(n):
        """Return random pure state density matrix"""
        rho = rand_vec(n, normalize=True)
        return np.outer(rho, np.conjugate(rho))

def assertEqual(a, b):
    if isinstance(a, np.ndarray):
        assert np.all(a == b)
    else:
        assert a == b

def assertNotEqual(a, b):
    assert a != b

def assertTrue(a):
    assertEqual(a, True)

def assertFalse(a):
    assertEqual(a, False)

def assertIsNone(a):
    assertTrue(a is None)

def assertIn(a, b):
    assertTrue(a in b)

def assertRaises(exception, func, *args, **kwds):
    try:
        func(*args, **kwds)
    except exception:
        pass

def assertAlmostEqual(a, b):
    assertTrue(np.isclose(a, b))

# https://stackoverflow.com/questions/23549419/assert-that-two-dictionaries-are-almost-equal
def _assertDictAlmostEqual(dict1, dict2, rel_tol=1e-8):
    """
    If dictionary value is a number, then check that the numbers are almost equal, otherwise check if values are exactly equal
    Note: does not currently try converting strings to digits and comparing them. Does not care about ordering of keys in dictionaries
    Just returns true or false
    """
    if len(dict1) != len(dict2):
        return False
    # Loop through each item in the first dict and compare it to the second dict
    for key, item in dict1.items():
        # If it is a nested dictionary, need to call the function again
        if isinstance(item, dict):
            # If the nested dictionaries are not almost equal, return False
            if not _assertDictAlmostEqual(dict1[key], dict2[key], rel_tol=rel_tol):
                return False
        # If it's not a dictionary, then continue comparing
        # Put in else statement or else the nested dictionary will get compared twice and
        # On the second time will check for exactly equal and will fail
        else:
            # If the value is a number, check if they are approximately equal
            if isinstance(item, Number):
                # if not abs(dict1[key] - dict2[key]) <= rel_tol:
                # https://stackoverflow.com/questions/5595425/what-is-the-best-way-to-compare-floats-for-almost-equality-in-python
                if hasattr(item, 'imag'):
                    if not math.isclose(dict1[key].real, dict2[key].real, rel_tol=rel_tol) or not math.isclose(dict1[key].imag, dict2[key].imag, rel_tol=rel_tol):
                        return False
                else:
                    if not math.isclose(dict1[key], dict2[key], rel_tol=rel_tol):
                        return False
            else:
                if not (dict1[key] == dict2[key]):
                    return False
    return True

def assertDictAlmostEqual(dict1, dict2, rel_tol=1e-8):
    assertTrue(_assertDictAlmostEqual(dict1, dict2, rel_tol))

@contextlib.contextmanager
def subTest(**kwds):
    try:
        yield
    except Exception as e:
        msg = kwds['msg'] if 'msg' in kwds else ''
        print(e, msg, file=sys.stderr)

In [ ]:
from qiskit.quantum_info.states.random import _random_density_hs, _random_density_bures

def random_density_matrix(dims, rank=None, method="Hilbert-Schmidt", seed=None):
    """Generator a random AerDensityMatrix.
    Args:
        dims (int or tuple): the dimensions of the AerDensityMatrix.
        rank (int or None): Optional, the rank of the density matrix.
                            The default value is full-rank.
        method (string): Optional. The method to use.
            'Hilbert-Schmidt': (Default) sample from the Hilbert-Schmidt metric.
            'Bures': sample from the Bures metric.
        seed (int or np.random.Generator): Optional. Set a fixed seed or
                                           generator for RNG.
    Returns:
        AerDensityMatrix: the random density matrix.
    Raises:
        QiskitError: if the method is not valid.
    """
    # Flatten dimensions
    dim = np.product(dims)
    if rank is None:
        rank = dim  # Use full rank

    if method == "Hilbert-Schmidt":
        rho = _random_density_hs(dim, rank, seed)
    elif method == "Bures":
        rho = _random_density_bures(dim, rank, seed)
    else:
        raise QiskitError(f"Error: unrecognized method {method}")
    return AerDensityMatrix(rho, dims=dims)

In [ ]:
arr = np.array([[1, 2],
                [3, 4]])
for num_array in np.nditer(arr, order='F'):
    print(num_array, end=' ')

In [ ]:
print(np.sum([np.abs(a)**2 for a in rand_vec(3, True)]))
print(np.trace(rand_rho(3)))

In [ ]:
state = np.array([[1/2, np.sqrt(3)/2*1j]])
assert math.isclose(np.sum(state * state.conj()).real, 1)
assert math.isclose(np.sum(state * state.conj()).imag, 0)

rho = state.reshape(-1, 1) @ state.conj()
assert np.all(rho == np.outer(state, state.conj()))

assert math.isclose(rho.trace().real, 1)
assert math.isclose(rho.trace().imag, 0)

state1 = DensityMatrix(rho)
state2 = AerDensityMatrix(rho)
assert_allclose(state1.data, rho)
assert_allclose(state1.data, state2.data)
assert_allclose(state2.data, rho)

In [ ]:
# XXX

u0 = random_unitary(2).data

# Direct calculation
sv = u0.dot([1, 0])
dm = np.outer(sv, sv.conj())

# Calculate via QuantumCircuit
qr = QuantumRegister(1)
circ = QuantumCircuit(qr)
circ.unitary(u0, [qr[0]])
rho = AerDensityMatrix(circ)

# Calculate from directly calculated density matrix
target = AerDensityMatrix(dm)

assertEqual(target.data, dm)
assertEqual(target, rho)

In [ ]:
for n in [2, 2**2, 2**3]:
    rho = rand_rho(n)
    for dims in [None, n]:
        state1 = DensityMatrix(rho, dims=dims)
        state2 = AerDensityMatrix(rho, dims=dims)
        assert_allclose(state1.data, rho)
        assert_allclose(state1.data, state2.data)

In [ ]:
"""Test generation of Aer's DensityMatrix with QV """
circ = QuantumVolume(5, seed=1111)
state = AerDensityMatrix(circ)
expected = DensityMatrix(circ)

for e, s in zip(expected.data, state.data):
    assertAlmostEqual(e, s)

In [ ]:
"""Test method and device properties"""
circ = QuantumVolume(5, seed=1111)

state1 = AerDensityMatrix(circ)
assertEqual('density_matrix', state1.metadata()['method'])
assertEqual('CPU', state1.metadata()['device'])

state2 = AerDensityMatrix(circ, method='density_matrix')
assertEqual('density_matrix', state2.metadata()['method'])
assertEqual('CPU', state2.metadata()['device'])

assertEqual(state1, state2)

In [ ]:
"""Test each method can process ghz"""
ghz = QuantumCircuit(4)
ghz.h(0)
ghz.cx(0, 1)
ghz.cx(1, 2)
ghz.cx(2, 3)

dm = AerDensityMatrix(ghz)
counts = dm.sample_counts(shots=1024)
assertEqual(2, len(counts))
assertTrue('0000' in counts)
assertTrue('1111' in counts)

In [ ]:
"""Test each method can process qft"""
qft = QuantumCircuit(4)
qft.h(range(4))
qft.compose(QFT(4), inplace=True)

dm = AerDensityMatrix(qft)
counts = dm.sample_counts(shots=1024)
assertEqual(1, len(counts))
assertTrue('0000' in counts)

In [ ]:
"""Test single qubit QuantumVolume"""
state = AerDensityMatrix(QuantumVolume(1))
counts = state.sample_counts(shots=1024)
assertEqual(1, len(counts))
assertTrue('0' in counts)

In [ ]:
"""Test evolve method for circuits"""
circ1 = QuantumVolume(5, seed=1111)
circ2 = circ1.compose(circ1)
circ3 = circ2.compose(circ1)

state1 = AerDensityMatrix(circ1)
state2 = AerDensityMatrix(circ2)
state3 = AerDensityMatrix(circ3)

assertEqual(state1.evolve(circ1), state2)
assertEqual(state1.evolve(circ1).evolve(circ1), state3)

In [ ]:
"""Test basic gates can be decomposed correctly"""
circ = QuantumCircuit(3)
circ.h(0)
circ.x(1)
circ.ry(np.pi / 2, 2)
state1 = AerDensityMatrix(circ)

In [ ]:
# Test tensor product of 1-qubit gates
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.x(1)
circuit.ry(np.pi / 2, 2)
psi = AerDensityMatrix.from_instruction(circuit)
target = AerDensityMatrix.from_label("000").evolve(Operator(circuit))
assertEqual(target, psi)

In [ ]:
# Test tensor product of 1-qubit gates
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.h(1)
target = AerDensityMatrix.from_label("000").evolve(Operator(circuit))
psi = AerDensityMatrix.from_instruction(circuit)
assertEqual(psi, target)

In [ ]:
"""Test deep copy"""
import copy
circ1 = QuantumVolume(5, seed=1111)

state1 = AerDensityMatrix(circ1)
state2 = copy.deepcopy(state1)

assertEqual(state1.data.shape, state2.data.shape)
for pa1, pa2 in zip(state1.data.ravel(), state2.data.ravel()):
    assertAlmostEqual(pa1, pa2)

assertNotEqual(id(state1._data), id(state2._data))

In [ ]:
"""Test ndarray initialization """
circ = QuantumVolume(5, seed=1111)
expected = DensityMatrix(circ)
state = AerDensityMatrix(expected.data)

for e, s in zip(expected.data, state.data):
    assertAlmostEqual(e, s)

In [ ]:
"""Test Statevector initialization """
circ = QuantumVolume(5, seed=1111)
sv = Statevector(circ)
expected = np.outer(sv, np.conjugate(sv))
state = AerDensityMatrix(sv)

for e, s in zip(expected, state.data):
    assertAlmostEqual(e, s)

In [ ]:
"""Test AerStatevector initialization """
circ = QuantumVolume(5, seed=1111)
sv = AerStatevector(circ)
expected = np.outer(sv, np.conjugate(sv))
state = AerDensityMatrix(sv)

for e, s in zip(expected, state.data):
    assertAlmostEqual(e, s)

In [ ]:
"""Test DensityMatrix initialization """
circ = QuantumVolume(5, seed=1111)
expected = DensityMatrix(circ)
state = AerDensityMatrix(expected)

for e, s in zip(expected.data, state.data):
    assertAlmostEqual(e, s)

####                                            ####
###   Copy from test_densitymatrix.py in terra   ###
####                                            ####

In [ ]:
"""Test subsystem initialization from N-qubit array."""
# Test automatic inference of qubit subsystems
rho = rand_rho(8)
for dims in [None, 8]:
    state = AerDensityMatrix(rho, dims=dims)
    assert_allclose(state.data, rho)
    assertEqual(state.dim, 8)
    assertEqual(state.dims(), (2, 2, 2))
    assertEqual(state.num_qubits, 3)

In [ ]:
if False:
    """Test initialization from array."""
    rho = rand_rho(3)
    state = AerDensityMatrix(rho)
    assert_allclose(state.data, rho)
    assertEqual(state.dim, 3)
    assertEqual(state.dims(), (3,))
    assertIsNone(state.num_qubits)

    rho = rand_rho(2 * 3 * 4)
    state = AerDensityMatrix(rho, dims=[2, 3, 4])
    assert_allclose(state.data, rho)
    assertEqual(state.dim, 2 * 3 * 4)
    assertEqual(state.dims(), (2, 3, 4))
    assertIsNone(state.num_qubits)

In [ ]:
"""Test initialization exception from array."""
rho = rand_rho(4)
assertRaises(QiskitError, AerDensityMatrix, rho, dims=[4, 2])
assertRaises(QiskitError, AerDensityMatrix, rho, dims=[2, 4])
assertRaises(QiskitError, AerDensityMatrix, rho, dims=5)

In [ ]:
"""Test initialization from AerDensityMatrix."""
rho1 = AerDensityMatrix(rand_rho(4))
rho2 = AerDensityMatrix(rho1)
assertEqual(rho1, rho2)

In [ ]:
"""Test initialization from AerDensityMatrix."""
vec = rand_vec(4)
target = AerDensityMatrix(np.outer(vec, np.conjugate(vec)))
rho = AerDensityMatrix(AerStatevector(vec))
assertEqual(rho, target)

In [ ]:
if False:
    from qiskit import transpile
    from qiskit_aer.backends.backend_utils import BASIS_GATES

    qc = QuantumCircuit(2)
    #qc.h(0)
    qc.unitary(random_unitary(2), [0])
    qc.cx(0, 1)

    display(qc.draw())

    t_qc = transpile(qc, basis_gates=BASIS_GATES['density_matrix'])
    display(t_qc.draw())
    print(BASIS_GATES['density_matrix'])

In [ ]:
zero = np.array([1, 0])
init_dm = np.outer(zero, zero.conj())
theta = np.pi/2
def hadamard():
    return np.array([[1, 1],
                     [1, -1]]) / np.sqrt(2)
def rz(theta):
    return np.array([[np.exp(-1j*theta/2), 0],
                     [0, np.exp(1j*theta/2)]])
# direct calc
final_dm = rz(theta)@hadamard()@init_dm@hadamard().conj().T@rz(theta).conj().T

# use qiskit.quantum_info.DensityMatrix
qc = QuantumCircuit(1)
qc.h(0)
qc.rz(theta, 0)
dm_terra = DensityMatrix(qc)
assertEqual(final_dm, dm_terra.data)

for instruction in qc.data:
    inst = instruction.operation
    definition = inst.definition
    qargs = instruction.qubits
    if definition is inst:
        print(inst.name, [qc.find_bit(qarg).index for qarg in qargs], instruction)
    else:
        print(definition.data)

dm = AerDensityMatrix(qc)

In [ ]:
"""Test initialization from a circuit."""
# random unitaries
u0 = random_unitary(2).data
u1 = random_unitary(2).data
# add to circuit
qr = QuantumRegister(2)
circ = QuantumCircuit(qr)
circ.unitary(u0, [qr[0]])
circ.unitary(u1, [qr[1]])
target_vec = Statevector(np.kron(u1, u0).dot([1, 0, 0, 0]))
target = DensityMatrix(target_vec)
rho = DensityMatrix(circ)
assertEqual(rho, target)

In [ ]:
x = np.array([
    [0., 1.],
    [1., 0.]
], dtype=np.complex128)
h = np.array([
    [1., 1.],
    [1., -1.]
], dtype=np.complex128) / np.sqrt(2)
p = lambda theta: np.array([
    [1., 0.],
    [0., np.exp(theta*1j)]
], dtype=np.complex128)

qr = QuantumRegister(1)
circ = QuantumCircuit(qr)
circ.unitary(x, [qr[0]])
circ.unitary(h, [qr[0]])
target_vec = AerStatevector((h@x).dot([1, 0]))
target = AerDensityMatrix(target_vec)
rho = AerDensityMatrix(circ)
assertEqual(rho, target)

In [ ]:
# XXX

u0 = random_unitary(2).data
qr = QuantumRegister(1)
circ = QuantumCircuit(qr)
circ.unitary(u0, [qr[0]])
target_vec = AerStatevector(u0.dot([1, 0]))
target = AerDensityMatrix(target_vec)
rho = AerDensityMatrix(circ)
assertEqual(rho.data, target.data)

In [ ]:
qr = QuantumRegister(2)
circ = QuantumCircuit(qr)
circ.unitary(x, [qr[0]])
circ.unitary(p(np.pi/4), [qr[1]])
circ.unitary(h, [qr[1]])
target_vec = AerStatevector(np.kron((h@p(np.pi/4)), x).dot([1, 0, 0, 0]))
target = AerDensityMatrix(target_vec)
rho = AerDensityMatrix(circ)
assertEqual(rho, target)
assertTrue(np.isclose(rho.data, target.data))

In [ ]:
"""Test initialization from a circuit."""
# random unitaries
u0 = random_unitary(2).data
u1 = random_unitary(2).data
# add to circuit
qr = QuantumRegister(2)
circ = QuantumCircuit(qr)
circ.unitary(u0, [qr[0]])
circ.unitary(u1, [qr[1]])
target_vec = AerStatevector(np.kron(u1, u0).dot([1, 0, 0, 0]))
target = AerDensityMatrix(target_vec)
rho = AerDensityMatrix(circ)
assertEqual(rho, target)

In [ ]:
"""Test initialization from a circuit."""
# random unitaries
u0 = random_unitary(2).data
u1 = random_unitary(2).data
# add to circuit
qr = QuantumRegister(2)
circ = QuantumCircuit(qr)
circ.unitary(u0, [qr[0]])
circ.unitary(u1, [qr[1]])

# Test decomposition of controlled-H gate
circuit = QuantumCircuit(2)
circ.x(0)
circuit.ch(0, 1)
target = AerDensityMatrix.from_label("00").evolve(Operator(circuit))
rho = AerDensityMatrix.from_instruction(circuit)
assertEqual(rho, target)

# Test initialize instruction
init = AerStatevector([1, 0, 0, 1j]) / np.sqrt(2)
target = AerDensityMatrix(init)
circuit = QuantumCircuit(2)
circuit.initialize(init.data, [0, 1])
rho = AerDensityMatrix.from_instruction(circuit)
assertEqual(rho, target)

# Test reset instruction
target = AerDensityMatrix([1, 0])
circuit = QuantumCircuit(1)
circuit.h(0)
circuit.reset(0)
rho = AerDensityMatrix.from_instruction(circuit)
assertEqual(rho, target)

In [ ]:
"""Test initialization from an instruction."""
target_vec = AerStatevector(np.dot(HGate().to_matrix(), [1, 0]))
target = AerDensityMatrix(target_vec)
rho = AerDensityMatrix.from_instruction(HGate())
assertEqual(rho, target)

In [ ]:
"""Test initialization from a label"""
x_p = AerDensityMatrix(np.array([[0.5, 0.5], [0.5, 0.5]]))
x_m = AerDensityMatrix(np.array([[0.5, -0.5], [-0.5, 0.5]]))
y_p = AerDensityMatrix(np.array([[0.5, -0.5j], [0.5j, 0.5]]))
y_m = AerDensityMatrix(np.array([[0.5, 0.5j], [-0.5j, 0.5]]))
z_p = AerDensityMatrix(np.diag([1, 0]))
z_m = AerDensityMatrix(np.diag([0, 1]))

label = "0+r"
target = z_p.tensor(x_p).tensor(y_p)
assertEqual(target, AerDensityMatrix.from_label(label))

label = "-l1"
target = x_m.tensor(y_m).tensor(z_m)
assertEqual(target, AerDensityMatrix.from_label(label))

In [ ]:
"""Test __eq__ method"""
for _ in range(10):
    rho = rand_rho(4)
    assertEqual(AerDensityMatrix(rho), AerDensityMatrix(rho.tolist()))

In [ ]:
"""Test AerDensityMatrix copy method"""
for _ in range(5):
    rho = rand_rho(4)
    orig = AerDensityMatrix(rho)
    cpy = orig.copy()
    cpy._data[0] += 1.0
    assertFalse(cpy == orig)

In [ ]:
"""Test is_valid method."""
state = AerDensityMatrix(np.eye(2))
assertFalse(state.is_valid())
for _ in range(10):
    state = AerDensityMatrix(rand_rho(4))
    assertTrue(state.is_valid())

In [ ]:
"""Test to_operator method for returning projector."""
for _ in range(10):
    rho = rand_rho(4)
    target = Operator(rho)
    op = AerDensityMatrix(rho).to_operator()
    assertEqual(op, target)

In [ ]:
"""Test evolve method for operators."""
for _ in range(10):
    op = random_unitary(4)
    rho = rand_rho(4)
    target = AerDensityMatrix(np.dot(op.data, rho).dot(op.adjoint().data))
    evolved = AerDensityMatrix(rho).evolve(op)
    assertEqual(target, evolved)

In [ ]:
"""Test subsystem evolve method for operators."""
# Test evolving single-qubit of 3-qubit system
for _ in range(5):
    rho = rand_rho(8)
    state = AerDensityMatrix(rho)
    op0 = random_unitary(2)
    op1 = random_unitary(2)
    op2 = random_unitary(2)

    # Test evolve on 1-qubit
    op = op0
    op_full = Operator(np.eye(4)).tensor(op)
    target = AerDensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[0]), target)

    # Evolve on qubit 1
    op_full = Operator(np.eye(2)).tensor(op).tensor(np.eye(2))
    target = AerDensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[1]), target)

    # Evolve on qubit 2
    op_full = op.tensor(np.eye(4))
    target = AerDensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[2]), target)

    # Test evolve on 2-qubits
    op = op1.tensor(op0)

    # Evolve on qubits [0, 2]
    op_full = op1.tensor(np.eye(2)).tensor(op0)
    target = AerDensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[0, 2]), target)

    # Evolve on qubits [2, 0]
    op_full = op0.tensor(np.eye(2)).tensor(op1)
    target = AerDensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[2, 0]), target)

    # Test evolve on 3-qubits
    op = op2.tensor(op1).tensor(op0)

    # Evolve on qubits [0, 1, 2]
    op_full = op
    target = AerDensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[0, 1, 2]), target)

    # Evolve on qubits [2, 1, 0]
    op_full = op0.tensor(op1).tensor(op2)
    target = AerDensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[2, 1, 0]), target)

In [ ]:
if False:
    """Test nested evolve calls on qudit subsystems."""
    dims = (3, 4, 5)
    init = self.rand_rho(np.prod(dims))
    ops = [random_unitary((dim,)) for dim in dims]
    state = AerDensityMatrix(init, dims)
    for i, op in enumerate(ops):
        state = state.evolve(op, [i])
    target_op = np.eye(1)
    for op in ops:
        target_op = np.kron(op.data, target_op)
    target = AerDensityMatrix(np.dot(target_op, init).dot(target_op.conj().T), dims)
    assertEqual(state, target)

In [ ]:
"""Test conjugate method."""
for _ in range(10):
    rho = rand_rho(4)
    target = AerDensityMatrix(np.conj(rho))
    state = AerDensityMatrix(rho).conjugate()
    assertEqual(state.data, target.data)

In [ ]:
"""Test expand method."""
for _ in range(10):
    rho0 = rand_rho(2)
    rho1 = rand_rho(3)
    target = np.kron(rho1, rho0)
    state = AerDensityMatrix(rho0).expand(AerDensityMatrix(rho1))
    assertEqual(state.dim, 6)
    assertEqual(state.dims(), (2, 3))
    assert_allclose(state.data, target)

In [ ]:
"""Test tensor method."""
for _ in range(10):
    rho0 = rand_rho(2)
    rho1 = rand_rho(3)
    target = np.kron(rho0, rho1)
    state = AerDensityMatrix(rho0).tensor(AerDensityMatrix(rho1))
    assertEqual(state.dim, 6)
    assertEqual(state.dims(), (3, 2))
    assert_allclose(state.data, target)

In [ ]:
"""Test add method."""
for _ in range(10):
    rho0 = rand_rho(4)
    rho1 = rand_rho(4)
    state0 = AerDensityMatrix(rho0)
    state1 = AerDensityMatrix(rho1)
    assertEqual(state0 + state1, AerDensityMatrix(rho0 + rho1))

In [ ]:
"""Test add method raises exceptions."""
state1 = AerDensityMatrix(rand_rho(2))
state2 = AerDensityMatrix(rand_rho(3))
assertRaises(QiskitError, state1.__add__, state2)

In [ ]:
"""Test subtract method."""
for _ in range(10):
    rho0 = rand_rho(4)
    rho1 = rand_rho(4)
    state0 = AerDensityMatrix(rho0)
    state1 = AerDensityMatrix(rho1)
    assertEqual(state0 - state1, AerDensityMatrix(rho0 - rho1))

In [ ]:
"""Test multiply method."""
for _ in range(10):
    rho = rand_rho(4)
    state = AerDensityMatrix(rho)
    val = np.random.rand() + 1j * np.random.rand()
    assertEqual(val * state, AerDensityMatrix(val * state))

In [ ]:
"""Test negate method"""
for _ in range(10):
    rho = rand_rho(4)
    state = AerDensityMatrix(rho)
    assertEqual(-state, AerDensityMatrix(-1 * rho))

In [ ]:
"""Test to_dict method"""

with subTest(msg="dims = (2, 2)"):
    rho = AerDensityMatrix(np.arange(1, 17).reshape(4, 4))
    target = {
        "00|00": 1,
        "01|00": 2,
        "10|00": 3,
        "11|00": 4,
        "00|01": 5,
        "01|01": 6,
        "10|01": 7,
        "11|01": 8,
        "00|10": 9,
        "01|10": 10,
        "10|10": 11,
        "11|10": 12,
        "00|11": 13,
        "01|11": 14,
        "10|11": 15,
        "11|11": 16,
    }
    assertDictAlmostEqual(target, rho.to_dict())

with subTest(msg="dims = (2, 3)"):
    rho = AerDensityMatrix(np.diag(np.arange(1, 7)), dims=(2, 3))
    target = {}
    for i in range(2):
        for j in range(3):
            key = "{1}{0}|{1}{0}".format(i, j)
            target[key] = 2 * j + i + 1
    assertDictAlmostEqual(target, rho.to_dict())

with subTest(msg="dims = (2, 11)"):
    vec = AerDensityMatrix(np.diag(np.arange(1, 23)), dims=(2, 11))
    target = {}
    for i in range(2):
        for j in range(11):
            key = "{1},{0}|{1},{0}".format(i, j)
            target[key] = 2 * j + i + 1
    assertDictAlmostEqual(target, vec.to_dict())

In [ ]:
"""Test converting a pure density matrix to AerStatevector."""
state = 1 / np.sqrt(2) * (np.array([1, 0, 0, 0, 0, 0, 0, 1]))
psi = AerStatevector(state)
rho = AerDensityMatrix(psi)
phi = rho.to_statevector()
assertTrue(psi.equiv(phi))

In [ ]:
"""Test converting a pure density matrix to AerStatevector."""
state_1 = 1 / np.sqrt(2) * (np.array([1, 0, 0, 0, 0, 0, 0, 1]))
state_2 = 1 / np.sqrt(2) * (np.array([0, 0, 0, 0, 0, 0, 1, 1]))
psi = 0.5 * (AerStatevector(state_1) + AerStatevector(state_2))
rho = AerDensityMatrix(psi)
assertRaises(QiskitError, rho.to_statevector)

In [ ]:
"""Test probabilities method for product state"""

state = AerDensityMatrix.from_label("+0")

# 2-qubit qargs
with subTest(msg="P(None)"):
    probs = state.probabilities()
    target = np.array([0.5, 0, 0.5, 0])
    assertTrue(np.allclose(probs, target))

with subTest(msg="P([0, 1])"):
    probs = state.probabilities([0, 1])
    target = np.array([0.5, 0, 0.5, 0])
    assertTrue(np.allclose(probs, target))

with subTest(msg="P([1, 0]"):
    probs = state.probabilities([1, 0])
    target = np.array([0.5, 0.5, 0, 0])
    assertTrue(np.allclose(probs, target))

# 1-qubit qargs
with subTest(msg="P([0])"):
    probs = state.probabilities([0])
    target = np.array([1, 0])
    assertTrue(np.allclose(probs, target))

with subTest(msg="P([1])"):
    probs = state.probabilities([1])
    target = np.array([0.5, 0.5])
    assertTrue(np.allclose(probs, target))

In [ ]:
"""Test probabilities method for GHZ state"""

psi = (AerStatevector.from_label("000") + AerStatevector.from_label("111")) / np.sqrt(2)
state = AerDensityMatrix(psi)

# 3-qubit qargs
target = np.array([0.5, 0, 0, 0, 0, 0, 0, 0.5])
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities(qargs)
        assertTrue(np.allclose(probs, target))

# 2-qubit qargs
target = np.array([0.5, 0, 0, 0.5])
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities(qargs)
        assertTrue(np.allclose(probs, target))

# 1-qubit qargs
target = np.array([0.5, 0.5])
for qargs in [[0], [1], [2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities(qargs)
        assertTrue(np.allclose(probs, target))

In [ ]:
"""Test probabilities method with W state"""

psi = (
    AerStatevector.from_label("001")
    + AerStatevector.from_label("010")
    + AerStatevector.from_label("100")
) / np.sqrt(3)
state = AerDensityMatrix(psi)

# 3-qubit qargs
target = np.array([0, 1 / 3, 1 / 3, 0, 1 / 3, 0, 0, 0])
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities(qargs)
        assertTrue(np.allclose(probs, target))

# 2-qubit qargs
target = np.array([1 / 3, 1 / 3, 1 / 3, 0])
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities(qargs)
        assertTrue(np.allclose(probs, target))

# 1-qubit qargs
target = np.array([2 / 3, 1 / 3])
for qargs in [[0], [1], [2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities(qargs)
        assertTrue(np.allclose(probs, target))

In [ ]:
"""Test probabilities_dict method for product state"""

state = AerDensityMatrix.from_label("+0")

# 2-qubit qargs
with subTest(msg="P(None)"):
    probs = state.probabilities_dict()
    target = {"00": 0.5, "10": 0.5}
    assertDictAlmostEqual(probs, target)

with subTest(msg="P([0, 1])"):
    probs = state.probabilities_dict([0, 1])
    target = {"00": 0.5, "10": 0.5}
    assertDictAlmostEqual(probs, target)

with subTest(msg="P([1, 0]"):
    probs = state.probabilities_dict([1, 0])
    target = {"00": 0.5, "01": 0.5}
    assertDictAlmostEqual(probs, target)

# 1-qubit qargs
with subTest(msg="P([0])"):
    probs = state.probabilities_dict([0])
    target = {"0": 1}
    assertDictAlmostEqual(probs, target)

with subTest(msg="P([1])"):
    probs = state.probabilities_dict([1])
    target = {"0": 0.5, "1": 0.5}
    assertDictAlmostEqual(probs, target)

In [ ]:
"""Test probabilities_dict method for GHZ state"""

psi = (AerStatevector.from_label("000") + AerStatevector.from_label("111")) / np.sqrt(2)
state = AerDensityMatrix(psi)

# 3-qubit qargs
target = {"000": 0.5, "111": 0.5}
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities_dict(qargs)
        assertDictAlmostEqual(probs, target)

# 2-qubit qargs
target = {"00": 0.5, "11": 0.5}
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities_dict(qargs)
        assertDictAlmostEqual(probs, target)

# 1-qubit qargs
target = {"0": 0.5, "1": 0.5}
for qargs in [[0], [1], [2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities_dict(qargs)
        assertDictAlmostEqual(probs, target)

In [ ]:
"""Test probabilities_dict method with W state"""

psi = (
    AerStatevector.from_label("001")
    + AerStatevector.from_label("010")
    + AerStatevector.from_label("100")
) / np.sqrt(3)
state = AerDensityMatrix(psi)

# 3-qubit qargs
target = np.array([0, 1 / 3, 1 / 3, 0, 1 / 3, 0, 0, 0])
target = {"001": 1 / 3, "010": 1 / 3, "100": 1 / 3}
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities_dict(qargs)
        assertDictAlmostEqual(probs, target)

# 2-qubit qargs
target = {"00": 1 / 3, "01": 1 / 3, "10": 1 / 3}
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities_dict(qargs)
        assertDictAlmostEqual(probs, target)

# 1-qubit qargs
target = {"0": 2 / 3, "1": 1 / 3}
for qargs in [[0], [1], [2]]:
    with subTest(msg=f"P({qargs})"):
        probs = state.probabilities_dict(qargs)
        assertDictAlmostEqual(probs, target)

In [ ]:
"""Test sample_counts method for GHZ state"""

shots = 2000
threshold = 0.02 * shots
state = AerDensityMatrix(
    (AerStatevector.from_label("000") + AerStatevector.from_label("111")) / np.sqrt(2)
)
state.seed(100)

# 3-qubit qargs
target = {"000": shots / 2, "111": shots / 2}
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:

    with subTest(msg=f"counts (qargs={qargs})"):
        counts = state.sample_counts(shots, qargs=qargs)
        assertDictAlmostEqual(counts, target, threshold)

# 2-qubit qargs
target = {"00": shots / 2, "11": shots / 2}
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:

    with subTest(msg=f"counts (qargs={qargs})"):
        counts = state.sample_counts(shots, qargs=qargs)
        assertDictAlmostEqual(counts, target, threshold)

# 1-qubit qargs
target = {"0": shots / 2, "1": shots / 2}
for qargs in [[0], [1], [2]]:

    with subTest(msg=f"counts (qargs={qargs})"):
        counts = state.sample_counts(shots, qargs=qargs)
        assertDictAlmostEqual(counts, target, threshold)

In [ ]:
"""Test sample_counts method for W state"""
shots = 3000
threshold = 0.02 * shots
state = AerDensityMatrix(
    (
        AerStatevector.from_label("001")
        + AerStatevector.from_label("010")
        + AerStatevector.from_label("100")
    )
    / np.sqrt(3)
)
state.seed(100)

target = {"001": shots / 3, "010": shots / 3, "100": shots / 3}
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:

    with subTest(msg=f"P({qargs})"):
        counts = state.sample_counts(shots, qargs=qargs)
        assertDictAlmostEqual(counts, target, threshold)

# 2-qubit qargs
target = {"00": shots / 3, "01": shots / 3, "10": shots / 3}
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:

    with subTest(msg=f"P({qargs})"):
        counts = state.sample_counts(shots, qargs=qargs)
        assertDictAlmostEqual(counts, target, threshold)

# 1-qubit qargs
target = {"0": 2 * shots / 3, "1": shots / 3}
for qargs in [[0], [1], [2]]:

    with subTest(msg=f"P({qargs})"):
        counts = state.sample_counts(shots, qargs=qargs)
        assertDictAlmostEqual(counts, target, threshold)

In [ ]:
"""Test sample_counts method for qutrit state"""
p = 0.3
shots = 1000
threshold = 0.03 * shots
state = AerDensityMatrix(np.diag([p, 0, 1 - p]))
state.seed(100)

with subTest(msg="counts"):
    target = {"0": shots * p, "2": shots * (1 - p)}
    counts = state.sample_counts(shots=shots)
    assertDictAlmostEqual(counts, target, threshold)

In [ ]:
"""Test sample_memory method for GHZ state"""

shots = 2000
state = AerDensityMatrix(
    (AerStatevector.from_label("000") + AerStatevector.from_label("111")) / np.sqrt(2)
)
state.seed(100)

# 3-qubit qargs
target = {"000": shots / 2, "111": shots / 2}
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:

    with subTest(msg=f"memory (qargs={qargs})"):
        memory = state.sample_memory(shots, qargs=qargs)
        assertEqual(len(memory), shots)
        assertEqual(set(memory), set(target))

# 2-qubit qargs
target = {"00": shots / 2, "11": shots / 2}
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:

    with subTest(msg=f"memory (qargs={qargs})"):
        memory = state.sample_memory(shots, qargs=qargs)
        assertEqual(len(memory), shots)
        assertEqual(set(memory), set(target))

# 1-qubit qargs
target = {"0": shots / 2, "1": shots / 2}
for qargs in [[0], [1], [2]]:

    with subTest(msg=f"memory (qargs={qargs})"):
        memory = state.sample_memory(shots, qargs=qargs)
        assertEqual(len(memory), shots)
        assertEqual(set(memory), set(target))

In [ ]:
"""Test sample_memory method for W state"""
shots = 3000
state = AerDensityMatrix(
    (
        AerStatevector.from_label("001")
        + AerStatevector.from_label("010")
        + AerStatevector.from_label("100")
    )
    / np.sqrt(3)
)
state.seed(100)

target = {"001": shots / 3, "010": shots / 3, "100": shots / 3}
for qargs in [[0, 1, 2], [2, 1, 0], [1, 2, 0], [1, 0, 2]]:

    with subTest(msg=f"memory (qargs={qargs})"):
        memory = state.sample_memory(shots, qargs=qargs)
        assertEqual(len(memory), shots)
        assertEqual(set(memory), set(target))

# 2-qubit qargs
target = {"00": shots / 3, "01": shots / 3, "10": shots / 3}
for qargs in [[0, 1], [2, 1], [1, 2], [1, 2]]:

    with subTest(msg=f"memory (qargs={qargs})"):
        memory = state.sample_memory(shots, qargs=qargs)
        assertEqual(len(memory), shots)
        assertEqual(set(memory), set(target))

# 1-qubit qargs
target = {"0": 2 * shots / 3, "1": shots / 3}
for qargs in [[0], [1], [2]]:

    with subTest(msg=f"memory (qargs={qargs})"):
        memory = state.sample_memory(shots, qargs=qargs)
        assertEqual(len(memory), shots)
        assertEqual(set(memory), set(target))

In [ ]:
"""Test sample_memory method for qutrit state"""
p = 0.3
shots = 1000
state = AerDensityMatrix(np.diag([p, 0, 1 - p]))
state.seed(100)

with subTest(msg="memory"):
    memory = state.sample_memory(shots)
    assertEqual(len(memory), shots)
    assertEqual(set(memory), {"0", "2"})

In [ ]:
"""Test reset method for 2-qubit state"""

state = AerDensityMatrix(np.diag([0.5, 0, 0, 0.5]))

with subTest(msg="reset"):
    rho = state.copy()
    value = rho.reset()
    target = AerDensityMatrix(np.diag([1, 0, 0, 0]))
    assertEqual(value, target)

with subTest(msg="reset"):
    rho = state.copy()
    value = rho.reset([0, 1])
    target = AerDensityMatrix(np.diag([1, 0, 0, 0]))
    assertEqual(value, target)

with subTest(msg="reset [0]"):
    rho = state.copy()
    value = rho.reset([0])
    target = AerDensityMatrix(np.diag([0.5, 0, 0.5, 0]))
    assertEqual(value, target)

with subTest(msg="reset [0]"):
    rho = state.copy()
    value = rho.reset([1])
    target = AerDensityMatrix(np.diag([0.5, 0.5, 0, 0]))
    assertEqual(value, target)

In [ ]:
"""Test reset method for qutrit"""

state = AerDensityMatrix(np.diag([1, 1, 1]) / 3)
state.seed(200)
value = state.reset()
target = AerDensityMatrix(np.diag([1, 0, 0]))
assertEqual(value, target)

In [ ]:
"""Test measure method for 2-qubit state"""

state = AerDensityMatrix.from_label("+0")
seed = 200
shots = 100

with subTest(msg="measure"):
    for i in range(shots):
        rho = state.copy()
        rho.seed(seed + i)
        outcome, value = rho.measure()
        assertIn(outcome, ["00", "10"])
        if outcome == "00":
            target = AerDensityMatrix.from_label("00")
            assertEqual(value, target)
        else:
            target = AerDensityMatrix.from_label("10")
            assertEqual(value, target)

with subTest(msg="measure [0, 1]"):
    for i in range(shots):
        rho = state.copy()
        outcome, value = rho.measure([0, 1])
        assertIn(outcome, ["00", "10"])
        if outcome == "00":
            target = AerDensityMatrix.from_label("00")
            assertEqual(value, target)
        else:
            target = AerDensityMatrix.from_label("10")
            assertEqual(value, target)

with subTest(msg="measure [1, 0]"):
    for i in range(shots):
        rho = state.copy()
        outcome, value = rho.measure([1, 0])
        assertIn(outcome, ["00", "01"])
        if outcome == "00":
            target = AerDensityMatrix.from_label("00")
            assertEqual(value, target)
        else:
            target = AerDensityMatrix.from_label("10")
            assertEqual(value, target)
with subTest(msg="measure [0]"):
    for i in range(shots):
        rho = state.copy()
        outcome, value = rho.measure([0])
        assertEqual(outcome, "0")
        target = AerDensityMatrix.from_label("+0")
        assertEqual(value, target)

with subTest(msg="measure [1]"):
    for i in range(shots):
        rho = state.copy()
        outcome, value = rho.measure([1])
        assertIn(outcome, ["0", "1"])
        if outcome == "0":
            target = AerDensityMatrix.from_label("00")
            assertEqual(value, target)
        else:
            target = AerDensityMatrix.from_label("10")
            assertEqual(value, target)

In [ ]:
"""Test measure method for qutrit"""

state = AerDensityMatrix(np.diag([1, 1, 1]) / 3)
seed = 200
shots = 100

for i in range(shots):
    rho = state.copy()
    rho.seed(seed + i)
    outcome, value = rho.measure()
    assertIn(outcome, ["0", "1", "2"])
    if outcome == "0":
        target = AerDensityMatrix(np.diag([1, 0, 0]))
        assertEqual(value, target)
    elif outcome == "1":
        target = AerDensityMatrix(np.diag([0, 1, 0]))
        assertEqual(value, target)
    else:
        target = AerDensityMatrix(np.diag([0, 0, 1]))
        assertEqual(value, target)

In [ ]:
"""Test from_int method"""

with subTest(msg="from_int(0, 4)"):
    target = AerDensityMatrix([1, 0, 0, 0])
    value = AerDensityMatrix.from_int(0, 4)
    assertEqual(target, value)

with subTest(msg="from_int(3, 4)"):
    target = AerDensityMatrix([0, 0, 0, 1])
    value = AerDensityMatrix.from_int(3, 4)
    assertEqual(target, value)

with subTest(msg="from_int(8, (3, 3))"):
    target = AerDensityMatrix([0, 0, 0, 0, 0, 0, 0, 0, 1], dims=(3, 3))
    value = AerDensityMatrix.from_int(8, (3, 3))
    assertEqual(target, value)

In [ ]:
"""Test expectation_value method"""

psi = AerStatevector([1, 0, 0, 1]) / np.sqrt(2)
rho = AerDensityMatrix(psi)
for label, target in [
    ("II", 1),
    ("XX", 1),
    ("YY", -1),
    ("ZZ", 1),
    ("IX", 0),
    ("YZ", 0),
    ("ZX", 0),
    ("YI", 0),
]:
    with subTest(msg=f"<{label}>"):
        op = Pauli(label)
        expval = rho.expectation_value(op)
        assertAlmostEqual(expval, target)

psi = AerStatevector([np.sqrt(2), 0, 0, 0, 0, 0, 0, 1 + 1j]) / 2
rho = AerDensityMatrix(psi)
for label, target in [
    ("XXX", np.sqrt(2) / 2),
    ("YYY", -np.sqrt(2) / 2),
    ("ZZZ", 0),
    ("XYZ", 0),
    ("YIY", 0),
]:
    with subTest(msg=f"<{label}>"):
        op = Pauli(label)
        expval = rho.expectation_value(op)
        assertAlmostEqual(expval, target)

labels = ["XXX", "IXI", "YYY", "III"]
coeffs = [3.0, 5.5, -1j, 23]
spp_op = SparsePauliOp.from_list(list(zip(labels, coeffs)))
expval = rho.expectation_value(spp_op)
target = 25.121320343559642 + 0.7071067811865476j
assertAlmostEqual(expval, target)

In [ ]:
for pauli in (
    "II",
    "IX",
    "IY",
    "IZ",
    "XI",
    "XX",
    "XY",
    "XZ",
    "YI",
    "YX",
    "YY",
    "YZ",
    "ZI",
    "ZX",
    "ZY",
    "ZZ",
    "-II",
    "-IX",
    "-IY",
    "-IZ",
    "-XI",
    "-XX",
    "-XY",
    "-XZ",
    "-YI",
    "-YX",
    "-YY",
    "-YZ",
    "-ZI",
    "-ZX",
    "-ZY",
    "-ZZ",
    "iII",
    "iIX",
    "iIY",
    "iIZ",
    "iXI",
    "iXX",
    "iXY",
    "iXZ",
    "iYI",
    "iYX",
    "iYY",
    "iYZ",
    "iZI",
    "iZX",
    "iZY",
    "iZZ",
    "-iII",
    "-iIX",
    "-iIY",
    "-iIZ",
    "-iXI",
    "-iXX",
    "-iXY",
    "-iXZ",
    "-iYI",
    "-iYX",
    "-iYY",
    "-iYZ",
    "-iZI",
    "-iZX",
    "-iZY",
    "-iZZ",
):
    """Test expectation_value method for Pauli op"""
    seed = 1020
    op = Pauli(pauli)
    rho = random_density_matrix(2**op.num_qubits, seed=seed)
    rho._data = np.reshape(rho.data.flatten(order="F"), rho.data.shape, order="F")
    target = rho.expectation_value(op.to_matrix())
    expval = rho.expectation_value(op)
    assertAlmostEqual(expval, target)

In [ ]:
for pauli in (
    "II",
    "IX",
    "IY",
    "IZ",
    "XI",
    "XX",
    "XY",
    "XZ",
    "YI",
    "YX",
    "YY",
    "YZ",
    "ZI",
    "ZX",
    "ZY",
    "ZZ",
    "-II",
    "-IX",
    "-IY",
    "-IZ",
    "-XI",
    "-XX",
    "-XY",
    "-XZ",
    "-YI",
    "-YX",
    "-YY",
    "-YZ",
    "-ZI",
    "-ZX",
    "-ZY",
    "-ZZ",
    "iII",
    "iIX",
    "iIY",
    "iIZ",
    "iXI",
    "iXX",
    "iXY",
    "iXZ",
    "iYI",
    "iYX",
    "iYY",
    "iYZ",
    "iZI",
    "iZX",
    "iZY",
    "iZZ",
    "-iII",
    "-iIX",
    "-iIY",
    "-iIZ",
    "-iXI",
    "-iXX",
    "-iXY",
    "-iXZ",
    "-iYI",
    "-iYX",
    "-iYY",
    "-iYZ",
    "-iZI",
    "-iZX",
    "-iZY",
    "-iZZ",
):
    """Test expectation_value method for Pauli op"""
    seed = 1020
    op = Pauli(pauli)
    rho = random_density_matrix(2**op.num_qubits, seed=seed)
    rho._data = np.reshape(rho.data.flatten(order="C"), rho.data.shape, order="C")
    target = rho.expectation_value(op.to_matrix())
    expval = rho.expectation_value(op)
    assertAlmostEqual(expval, target)

In [ ]:
for qubits in ([0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1]):
    """Test expectation_value method for Pauli op"""
    seed = 1020
    op = random_pauli(2, seed=seed)
    state = random_density_matrix(2**3, seed=seed)
    target = state.expectation_value(op.to_matrix(), qubits)
    expval = state.expectation_value(op, qubits)
    assertAlmostEqual(expval, target)

In [ ]:
"""Test reverse_qargs method"""
circ1 = QFT(5)
circ2 = circ1.reverse_bits()

state1 = AerDensityMatrix.from_instruction(circ1)
state2 = AerDensityMatrix.from_instruction(circ2)
assertEqual(state1.reverse_qargs(), state2)

In [ ]:
"""Test draw method"""
qc1 = QFT(5)
dm = AerDensityMatrix.from_instruction(qc1)
with subTest(msg="str(density_matrix)"):
    str(dm)
for drawtype in ["repr", "text", "latex", "latex_source", "qsphere", "hinton", "bloch"]:
    with subTest(msg=f"draw('{drawtype}')"):
        dm.draw(drawtype)